# Part 1

```

Suddenly, a scheduled job activates the system's disk defragmenter. Were the situation different, you might sit and watch it for a while, but today, you just don't have that kind of time. It's soaking up valuable system resources that are needed elsewhere, and so the only option is to help it finish its task as soon as possible.

The disk in question consists of a 128x128 grid; each square of the grid is either free or used. On this disk, the state of the grid is tracked by the bits in a sequence of knot hashes.

A total of 128 knot hashes are calculated, each corresponding to a single row in the grid; each hash contains 128 bits which correspond to individual grid squares. Each bit of a hash indicates whether that square is free (0) or used (1).

The hash inputs are a key string (your puzzle input), a dash, and a number from 0 to 127 corresponding to the row. For example, if your key string were flqrgnkx, then the first row would be given by the bits of the knot hash of flqrgnkx-0, the second row from the bits of the knot hash of flqrgnkx-1, and so on until the last row, flqrgnkx-127.

The output of a knot hash is traditionally represented by 32 hexadecimal digits; each of these digits correspond to 4 bits, for a total of 4 * 32 = 128 bits. To convert to bits, turn each hexadecimal digit to its equivalent binary value, high-bit first: 0 becomes 0000, 1 becomes 0001, e becomes 1110, f becomes 1111, and so on; a hash that begins with a0c2017... in hexadecimal would begin with 10100000110000100000000101110000... in binary.

Continuing this process, the first 8 rows and columns for key flqrgnkx appear as follows, using # to denote used squares, and . to denote free ones:

##.#.#..-->
.#.#.#.#   
....#.#.   
#.#.##.#   
.##.#...   
##..#..#   
.#...#..   
##.#.##.-->
|      |   
V      V   
In this example, 8108 squares are used across the entire 128x128 grid.

Given your actual key string, how many squares are used?
```

## Get knot hash algorithm from day 10

In [4]:
# Function that reverse a segment of length "length" 
# of array "data" starting at index "i"

def reverser(data,i,length):
    arr = data.copy()
    if length > len(arr):
        raise ValueError('length cannot be longer than list length')
    
    if length < len(arr)-i+1:
        segment = arr[i:i+length]
        arr[i:i+length] = segment[::-1]
        
    else:
        lenprewrap = len(arr[i:])
        lenpostwrap = len(arr[:length-len(arr)+i])
        segment = arr[i:]+arr[:length-len(arr)+i]
        segment = segment[::-1]
        arr[i:] = segment[:lenprewrap]
        arr[:length-len(arr)+i] = segment[lenprewrap:]

    return arr

In [5]:
def run_cycle(lengths,n):
    data = [x for x in range(256)]
    i = 0
    skipsize = 0
    for j in range(n):
        for length in lengths:
            data = reverser(data,i,length)
            skip = (length+skipsize)%len(data)
            if skip>len(data)-i:
                i = skip-(len(data)-i)
            else:
                i += skip
            skipsize += 1
    return data

In [6]:
def bitwiseor(arr):
    v = 0
    for a in arr:
        v = v^a
    return v

In [7]:
def knothash(s):
    lengths = [ord(x) for x in s] + [17, 31, 73, 47, 23]

    sparse = run_cycle(lengths,64)
    densehash = [bitwiseor(sparse[16*i:16*(i+1)]) for i in range(16)]
    densehex = ''.join(['{:02x}'.format(x) for x in densehash])

    # Modified to return 128 bit binary instead of hex
    return bin(int(densehex,16))[2:].zfill(128)


In [8]:
data = 'jzgqcdpd'

In [9]:
# List of lists with binary numbers as integers

arr = [list(knothash('{}-{}'.format(data,x))) for x in range(128)]

arr = [[int(y) for y in x] for x in arr]

In [10]:
# convert to numpy array
arr = np.array(arr)

In [11]:
arr.sum()

8074

```
Now, all the defragmenter needs to know is the number of regions. A region is a group of used squares that are all adjacent, not including diagonals. Every used square is in exactly one region: lone used squares form their own isolated regions, while several adjacent squares all count as a single region.

In the example above, the following nine regions are visible, each marked with a distinct digit:

11.2.3..-->
.1.2.3.4   
....5.6.   
7.8.55.9   
.88.5...   
88..5..8   
.8...8..   
88.8.88.-->
|      |   
V      V   
Of particular interest is the region marked 8; while it does not appear contiguous in this small view, all of the squares marked 8 are connected when considering the whole 128x128 grid. In total, in this example, 1242 regions are present.

How many regions are present given your key string?
```

In [13]:
# From image processing, I know a function
# that pulls out objects from a binary array

from scipy.ndimage.measurements import label

In [14]:
mask,n = label(arr)

In [15]:
n

1212